In [1]:
from loader_ref import ExcelLoader

In [2]:
from language_ref import Language

In [3]:
loader=ExcelLoader("/workspaces/de.uke.iam.automapping/src/automapping/VM_Fragebögen_HCHS_10000_20210708.xlsx", 'Kurzname', 'Langname')

In [4]:
sample = loader.load(Language.GERMAN)[0]

In [5]:
from translator_ref import HuggingFace

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_translator = HuggingFace(Language.GERMAN, Language.ENGLISH)

In [7]:
sample = model_translator.translate(sample)

In [8]:
print(sample)

Sample(id='E0', content='Have you been feeding yourself meatless in the past 12 months?', language=<Language.ENGLISH: 'en'>, concepts=[])


In [9]:
from preprocessor_ref import SpacyPreprocessor

In [10]:
model_spacy=SpacyPreprocessor(["lowercase", "stopwords", "punctuation", "lemmatization"])

In [11]:
model_spacy.transform(sample)

Sample(id='E0', content='feeding meatless past 12 months', language=<Language.ENGLISH: 'en'>, concepts=[])

In [12]:
sample

Sample(id='E0', content='feeding meatless past 12 months', language=<Language.ENGLISH: 'en'>, concepts=[])

## Concepts

In [13]:
from concepts_ref import Concepts

In [14]:
import pandas as pd

In [15]:
concepts = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/CONCEPT.csv', on_bad_lines="skip", delimiter="\t", low_memory=False)

In [16]:
synonyms = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/CONCEPT_SYNONYM.csv', on_bad_lines="skip", delimiter="\t", low_memory=False)

In [17]:
vocabulary = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/VOCABULARY.csv',on_bad_lines='skip', delimiter='\t')

In [18]:
concepts = Concepts.concatenate_concept_with_their_synonyms(concepts, synonyms, vocabulary, 'SNOMED')


/workspaces/de.uke.iam.automapping/src/automapping_ref/concepts_ref.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concepts["concept_name"] = (


In [19]:
from mapper_ref import TfIdf

In [20]:
model_tfidf = TfIdf(concepts)


In [21]:
sample = model_tfidf(sample)

In [22]:
sample

Sample(id='E0', content='feeding meatless past 12 months', language=<Language.ENGLISH: 'en'>, concepts=[Prediction(concept=Concept(name='past', concept_id=4132507, concept_code='410513005', domain_id='Observation', voc_version='2020-07-31 SNOMED CT International Edition; 2020-09-01 SNOMED CT US Edition; 2020-10-28 SNOMED CT UK Edition'), score=0.5975503830827499), Prediction(concept=Concept(name='infant feeding at 4 months', concept_id=4217242, concept_code='417489001', domain_id='Observation', voc_version='2020-07-31 SNOMED CT International Edition; 2020-09-01 SNOMED CT US Edition; 2020-10-28 SNOMED CT UK Edition'), score=0.5100337248064811), Prediction(concept=Concept(name='infant feeding at 9 months', concept_id=4014595, concept_code='169990005', domain_id='Synonym', voc_version='2020-07-31 SNOMED CT International Edition; 2020-09-01 SNOMED CT US Edition; 2020-10-28 SNOMED CT UK Edition'), score=0.5100337248064811), Prediction(concept=Concept(name='infant feeding at 6 months', conce

In [23]:
from detection_ref import Predictions

In [24]:
predictions = Predictions(sample)

In [25]:
Predictions.to_df(predictions, 5)

,SourceID,SourceName,targetConceptName,targetConceptID,targetConceptCode,targetDomainID,targetVocabularyVersion,MatchScore
0,E0,feeding meatless past 12 months,past,4132507,410513005,Observation,2020-07-31 SNOMED CT International Edition; 20...,0.597550
1,E0,feeding meatless past 12 months,infant feeding at 4 months,4217242,417489001,Observation,2020-07-31 SNOMED CT International Edition; 20...,0.510034
2,E0,feeding meatless past 12 months,infant feeding at 9 months,4014595,169990005,Synonym,2020-07-31 SNOMED CT International Edition; 20...,0.510034
3,E0,feeding meatless past 12 months,infant feeding at 6 months,4014725,169983000,Synonym,2020-07-31 SNOMED CT International Edition; 20...,0.510034
4,E0,feeding meatless past 12 months,infant feeding at 3 months,4016066,169977003,Synonym,2020-07-31 SNOMED CT International Edition; 20...,0.510034
